<div class="alert alert-block alert-info">
     <b>Import Modules Needed</b> <i>(need pandas >= 1.2 and python >= 3.9.0 for all the data cleaning techniques to work)</i>
</div>

In [1]:
import pandas as  pd
import os
import sweetviz as sv
import numpy as np
import dtale
import cudf
import dask_cudf

#!pip install git+https://github.com/innovationOUtside/fstring-magic.git

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [2]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>Find CSV Files in working dir</b>
</div>

In [3]:
csvStore = os.path.join(os.getcwd(),'__csvStore' )
csvlist = []
for file in enumerate(os.listdir(csvStore)):
    if file[1].endswith(".csv"):
        csvlist.append(os.path.join(str(csvStore), file[1]))
        print(os.path.join(str(csvStore), file[1]))

/home/derek/Jupyter_Notebooks/__csvStore/df_visits.csv
/home/derek/Jupyter_Notebooks/__csvStore/df_episode.csv
/home/derek/Jupyter_Notebooks/__csvStore/final.csv
/home/derek/Jupyter_Notebooks/__csvStore/df_OASIS.csv
/home/derek/Jupyter_Notebooks/__csvStore/dead_patients_final.csv
/home/derek/Jupyter_Notebooks/__csvStore/orig_data2.csv
/home/derek/Jupyter_Notebooks/__csvStore/eliminated.csv
/home/derek/Jupyter_Notebooks/__csvStore/orig_data.csv
/home/derek/Jupyter_Notebooks/__csvStore/cleaned_data.csv
/home/derek/Jupyter_Notebooks/__csvStore/event_mapping_df.csv
/home/derek/Jupyter_Notebooks/__csvStore/soc2.csv
/home/derek/Jupyter_Notebooks/__csvStore/du.csv
/home/derek/Jupyter_Notebooks/__csvStore/patients.csv
/home/derek/Jupyter_Notebooks/__csvStore/duplicate_ceo_ids.csv
/home/derek/Jupyter_Notebooks/__csvStore/soc.csv
/home/derek/Jupyter_Notebooks/__csvStore/alive_patients_final.csv
/home/derek/Jupyter_Notebooks/__csvStore/dv.csv
/home/derek/Jupyter_Notebooks/__csvStore/deduped_compl

In [4]:
list(enumerate(csvlist))

[(0, '/home/derek/Jupyter_Notebooks/__csvStore/df_visits.csv'),
 (1, '/home/derek/Jupyter_Notebooks/__csvStore/df_episode.csv'),
 (2, '/home/derek/Jupyter_Notebooks/__csvStore/final.csv'),
 (3, '/home/derek/Jupyter_Notebooks/__csvStore/df_OASIS.csv'),
 (4, '/home/derek/Jupyter_Notebooks/__csvStore/dead_patients_final.csv'),
 (5, '/home/derek/Jupyter_Notebooks/__csvStore/orig_data2.csv'),
 (6, '/home/derek/Jupyter_Notebooks/__csvStore/eliminated.csv'),
 (7, '/home/derek/Jupyter_Notebooks/__csvStore/orig_data.csv'),
 (8, '/home/derek/Jupyter_Notebooks/__csvStore/cleaned_data.csv'),
 (9, '/home/derek/Jupyter_Notebooks/__csvStore/event_mapping_df.csv'),
 (10, '/home/derek/Jupyter_Notebooks/__csvStore/soc2.csv'),
 (11, '/home/derek/Jupyter_Notebooks/__csvStore/du.csv'),
 (12, '/home/derek/Jupyter_Notebooks/__csvStore/patients.csv'),
 (13, '/home/derek/Jupyter_Notebooks/__csvStore/duplicate_ceo_ids.csv'),
 (14, '/home/derek/Jupyter_Notebooks/__csvStore/soc.csv'),
 (15, '/home/derek/Jupyter_N

<div class="alert alert-block alert-info">
    <b>Import the raw data</b>
</div>

In [5]:
df = pd.read_csv(csvlist[2])#.iloc[: , 2:]


In [6]:
dtale.show(df)

<div class="alert alert-block alert-info">
    <b>Find Columns that contain mixed data types to investigate for possible data quality issues</b>
</div>


In [7]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

<div class="alert alert-block alert-info">
    <b>Find Columns with Dates and convert them to datetime type</b>
</div>

In [8]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OFEPISODE', case=False))] 
df[cols_w_dates] = df[cols_w_dates].apply(pd.to_datetime)
cols_w_dates


Index(['epi_SocDate', 'epi_StartOfEpisode', 'epi_EndOfEpisode',
       'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT', 'M0090_INFO_COMPLETED_DT',
       'cedd_dateofdeath'],
      dtype='object')

<div class="alert alert-block alert-info">
    <b>Break up columns</b>
</div>

In [9]:
# Descriptive IDs
df.columns[:27]
colsMeta = df.columns[:27]
# OASIS Factors
df.columns[27:]
colsOASIS = df.columns[27:]

Index(['DispositionId', 'pa_id', 'mortality', 'year_born', 'pa_gender',
       'epi_id', 'ceo_id', 'epi_SocDate', 'epi_StartOfEpisode',
       'epi_EndOfEpisode', 'epi_DischargeDate', 'M0906_DC_TRAN_DTH_DT',
       'M0090_INFO_COMPLETED_DT', 'cedd_dateofdeath', 'DaysToDeath',
       'epi_AdmitType', 'dcc_desc', 'epi_dcrid', 'dcr_desc', 'ceo_HIPPS',
       'ceo_HHRG', 'ceas_ascode', 'icd_code', 'icd_code_description',
       'DiagnosisCategory', 'RoughAge', 'ceo_OASIScorrectionnum'],
      dtype='object')

Index(['M0100_ASSMT_REASON', 'M0110_EPISODE_TIMING', 'M0140_ETHNIC_AI_AN',
       'M0140_ETHNIC_ASIAN', 'M0140_ETHNIC_BLACK', 'M0140_ETHNIC_HISP',
       'M0140_ETHNIC_NH_PI', 'M0140_ETHNIC_WHITE', 'M1000_DC_IPPS_14_DA',
       'M1000_DC_IRF_14_DA', 'M1000_DC_LTC_14_DA', 'M1000_DC_LTCH_14_DA',
       'M1000_DC_NONE_14_DA', 'M1000_DC_OTH_14_DA', 'M1000_DC_PSYCH_14_DA',
       'M1000_DC_SNF_14_DA', 'M1028_ACTV_DIAG_DM', 'M1028_ACTV_DIAG_NOA',
       'M1028_ACTV_DIAG_PVD_PAD', 'M1030_THH_ENT_NUTRITION',
       'M1030_THH_IV_INFUSION', 'M1030_THH_NONE_ABOVE',
       'M1030_THH_PAR_NUTRITION', 'M1033_HOSP_RISK_5PLUS_MDCTN',
       'M1033_HOSP_RISK_COMPLIANCE', 'M1033_HOSP_RISK_CRNT_EXHSTN',
       'M1033_HOSP_RISK_HSTRY_FALLS', 'M1033_HOSP_RISK_MLTPL_ED_VISIT',
       'M1033_HOSP_RISK_MLTPL_HOSPZTN', 'M1033_HOSP_RISK_MNTL_BHV_DCLN',
       'M1033_HOSP_RISK_NONE_ABOVE', 'M1033_HOSP_RISK_OTHR_RISK',
       'M1033_HOSP_RISK_WEIGHT_LOSS', 'M1051_PVX_RCVD_AGNCY', 'M1060_HEIGHT_A',
       'M1060_

<div class="alert alert-block alert-info">
    <b>Important Summary Stats</b>
</div>

In [10]:
f"There are {df.groupby('DispositionId').agg({'pa_id': 'nunique'}).sum().values[0]} patients"

f"Alive: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[0,0]} | Dead: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[1,0]}"

'There are 3128 patients'

'Alive: 225 | Dead: 2903'

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-info" role="alert">
    <b>Create a patients df and populate the death date by taking the most recent dates</b>
</div>

In [11]:
len(df.groupby(['pa_id'])['pa_id','DispositionId','pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality'].transform('max').drop_duplicates('pa_id'))

df_patients = df.groupby(['pa_id'])['pa_id', 'DispositionId' , 'pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality', 'cedd_dateofdeath'].transform('max').drop_duplicates('pa_id').reset_index(drop=True)

3128

<div class="alert alert-info" role="alert">
    <b>If no cedd_dateofdeath is provided then use the 'M0906_DC_TRAN_DTH_DT' </b>
</div>

In [12]:
df_patients['cedd_dateofdeath'].where(((df_patients.mortality == 0) | (~df_patients.cedd_dateofdeath.isnull())), (df_patients['M0906_DC_TRAN_DTH_DT']), inplace=True)

<div class="alert alert-info" role="alert">
    <b>Create a Year of Death + Age at Death column</b>
</div>

In [13]:
df_patients['yod']= pd.DatetimeIndex(df_patients['cedd_dateofdeath']).year
df_patients['aod'] = df_patients['yod'] - df_patients['year_born']


<div class="alert alert-info" role="alert">
    <b>Convert YOD and AAD into integers</b>
</div>

In [14]:
#df_patients.fillna({'cedd_dateofdeath':0,}, inplace=True)
df_patients.fillna({'yod':0,}, inplace=True)
df_patients["yod"] = df_patients["yod"].apply(np.int64)
df_patients.fillna({'aod':0,}, inplace=True)
df_patients['aod'] = df_patients['aod'].apply(np.int64)

In [15]:
dtale.show(df_patients)

<div class="alert alert-info" role="alert">
    <b>Export df_patients to .csv</b>
</div>

In [16]:
df_patients.to_csv(csvStore+'/patients.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-warning" role="alert">
    <b>Find Best Columns to Use for Event Data </b>
</div>

In [17]:
(df.loc[:,colsOASIS][df['mortality']==0].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==0])).sort_values(ascending=False)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

M1060_WEIGHT_B                    0.866373
M1060_HEIGHT_A                    0.859031
M1730_PHQ2_LACK_INTRST            0.822320
M1730_PHQ2_DPRSN                  0.822320
M1028_ACTV_DIAG_DM                0.812041
M1028_ACTV_DIAG_NOA               0.812041
M1028_ACTV_DIAG_PVD_PAD           0.812041
M0140_ETHNIC_WHITE                0.810573
M0140_ETHNIC_HISP                 0.810573
M0140_ETHNIC_AI_AN                0.810573
M1910_MLT_FCTR_FALL_RISK_ASMT     0.810573
M2001_DRUG_RGMN_RVW               0.810573
M2010_HIGH_RISK_DRUG_EDCTN        0.810573
M1730_STDZ_DPRSN_SCRNG            0.810573
M1000_DC_IPPS_14_DA               0.810573
M0140_ETHNIC_BLACK                0.810573
M0140_ETHNIC_ASIAN                0.810573
M0140_ETHNIC_NH_PI                0.810573
M1000_DC_NONE_14_DA               0.810573
M1000_DC_IRF_14_DA                0.810573
M1000_DC_SNF_14_DA                0.810573
M1000_DC_PSYCH_14_DA              0.810573
M1000_DC_OTH_14_DA                0.810573
M1000_DC_LT

In [18]:
(df.loc[:,colsOASIS][df['mortality']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==1])).sort_values(ascending=True)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

M0100_ASSMT_REASON                0.000000
M1400_WHEN_DYSPNEIC               0.270668
M1306_UNHLD_STG2_PRSR_ULCR        0.270668
M1810_CRNT_DRESS_UPPER            0.270668
M1820_CRNT_DRESS_LOWER            0.270668
M1830_CRNT_BATHG                  0.270668
M1840_CRNT_TOILTG                 0.270668
M1242_PAIN_FREQ_ACTVTY_MVMT       0.270668
M1850_CRNT_TRNSFRNG               0.270668
M1860_CRNT_AMBLTN                 0.270668
M1324_STG_PRBLM_ULCER             0.270895
M1330_STAS_ULCR_PRSNT             0.270895
M1340_SRGCL_WND_PRSNT             0.270895
M1620_BWL_INCONT                  0.271234
M1800_CRNT_GROOMING               0.284258
M2102_CARE_TYPE_SRC_SPRVSN        0.420951
M1700_COG_FUNCTION                0.420951
M1710_WHEN_CONFUSED               0.420951
M1720_WHEN_ANXIOUS                0.420951
M1740_BD_IMP_DECISN               0.420951
M1845_CRNT_TOILTG_HYGN            0.420951
M1740_BD_MEM_DEFICIT              0.420951
M1740_BD_NONE                     0.420951
M1740_BD_PH

In [19]:
(df.loc[:,colsOASIS].isnull().sum()/len(df.loc[:,colsOASIS])).sort_values(ascending=True)

M0100_ASSMT_REASON                0.000000
M1400_WHEN_DYSPNEIC               0.272106
M1242_PAIN_FREQ_ACTVTY_MVMT       0.272106
M1810_CRNT_DRESS_UPPER            0.272106
M1820_CRNT_DRESS_LOWER            0.272106
M1830_CRNT_BATHG                  0.272106
M1840_CRNT_TOILTG                 0.272106
M1306_UNHLD_STG2_PRSR_ULCR        0.272106
M1860_CRNT_AMBLTN                 0.272106
M1850_CRNT_TRNSFRNG               0.272106
M1340_SRGCL_WND_PRSNT             0.272316
M1324_STG_PRBLM_ULCER             0.272316
M1330_STAS_ULCR_PRSNT             0.272316
M1620_BWL_INCONT                  0.272632
M1800_CRNT_GROOMING               0.288613
M1030_THH_PAR_NUTRITION           0.433183
M1030_THH_NONE_ABOVE              0.433183
M1030_THH_IV_INFUSION             0.433183
M1630_OSTOMY                      0.433183
M1030_THH_ENT_NUTRITION           0.433183
M1200_VISION                      0.433183
M2200_THER_NEED_NA                0.433183
M1322_NBR_PRSULC_STG1             0.433393
M2030_CRNT_

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
    <b>Create an events dataframe</b>
</div>

In [20]:
cols_Oevents = ['M0100_ASSMT_REASON', 
                'M1860_CRNT_AMBLTN', 
                'M1850_CRNT_TRNSFRNG', 
                'M1830_CRNT_BATHG', 
                'M1820_CRNT_DRESS_LOWER', 
                'M1810_CRNT_DRESS_UPPER', 
                'M1306_UNHLD_STG2_PRSR_ULCR', 
                'M1242_PAIN_FREQ_ACTVTY_MVMT', 
                'M1400_WHEN_DYSPNEIC', 
                'M1340_SRGCL_WND_PRSNT', 
                'M1330_STAS_ULCR_PRSNT',
                'M1324_STG_PRBLM_ULCER', 
                'M1620_BWL_INCONT', 
                'M1800_CRNT_GROOMING',
                'M1030_THH_PAR_NUTRITION',
                'M1030_THH_NONE_ABOVE',
                'M1030_THH_IV_INFUSION',
                'M1630_OSTOMY',
                'M1030_THH_ENT_NUTRITION',
                'M1200_VISION',
                'M2200_THER_NEED_NA',
                'M1322_NBR_PRSULC_STG1']

In [21]:
cols_Oevents = colsOASIS.tolist()

<div class="alert alert-success" role="alert">
    <b>Create an Items Mapping</b>
</div>

In [274]:
event_mapping_df = pd.DataFrame(cols_Oevents,columns =['OASIS']).rename_axis('itemID').reset_index(inplace=False)
event_mapping_df.itemID = np.arange(1, len(event_mapping_df) + 1)

In [275]:
dtale.show(event_mapping_df)

In [276]:
event_mapping_df.to_csv(csvStore+'/event_mapping_df.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-danger" role="alert">
    <b>View the Event Data in the Features of Interest</b>
</div>

In [277]:
df_events_int = df[cols_Oevents].reset_index()

In [278]:
dtale.show(df_events_int.head())

<div class="alert alert-danger" role="alert">
    <b>View the Unique Values in a particular Feature Column</b>
</div>

In [279]:
df_events_int.loc[:, 'M0110_EPISODE_TIMING'].unique()

array(['LATER', nan, 'EARLY', 'NOT APPLICABLE'], dtype=object)

<div class="alert alert-danger" role="alert">
    <b>View All Values of Enumerated Feature Columns</b>
</div>

In [280]:
for idx, col in enumerate(df_events_int):
    (idx, df_events_int.loc[:, col].unique())

(0, array([   0,    1,    2, ..., 9508, 9509, 9510]))

(1,
 array(['4 - RECERTIFICATION (FOLLOW-UP) REASSESSMENT',
        '9 - DISCHARGE FROM AGENCY',
        '1 - START OF CARE - FURTHER VISITS PLANNED',
        '6 - TRANSFERRED TO AN INPATIENT FACILITY - PATIENT NOT DISCHARGED FROM AGENCY',
        '8 - DEATH AT HOME',
        '7 - TRANSFERRED TO AN INPATIENT FACILITY - PATIENT DISCHARGED FROM AGENCY',
        '3 - RESUMPTION OF CARE (AFTER INPATIENT STAY)',
        '5 - OTHER FOLLOW-UP'], dtype=object))

(2, array(['LATER', nan, 'EARLY', 'NOT APPLICABLE'], dtype=object))

(3, array([nan, 'NO', 'YES'], dtype=object))

(4, array([nan, 'NO', 'YES'], dtype=object))

...

(85,
 array([nan, '1 - YES', '0 - NO',
        'NA - (C2) EVERY STANDARDIZED, VALIDATED MULTI-FACTOR FALL RISK ASSESSMENT CONDUCTED AT OR SINCE THE MOST RECENT SOC/ROC ASSESSMENT INDICATES THE PATIENT HAS NO RISK FOR FALLS.'],
       dtype=object))

(86,
 array([nan, '1 - YES',
        'NA - (C2) EVERY STANDARDIZED, VALIDATED PAIN ASSESSMENT CONDUCTED AT OR SINCE THE MOST RECENT SOC/ROC ASSESSMENT INDICATES THE PATIENT HAS NO PAIN.',
        '0 - NO'], dtype=object))

(87,
 array([nan, '1 - YES',
        'NA - (C2) EVERY STANDARDIZED, VALIDATED PRESSURE ULCER RISK ASSESSMENT CONDUCTED AT OR SINCE THE RECENT SOC/ROC  ASSESSMENT INDICATES THE PATIENT IS NOT AT RISK OF DEVELOPING PRESSURE ULCERS.',
        '0 - NO'], dtype=object))

(88,
 array([nan, '0 - NO', '1 - YES',
        'NA - PATIENT HAS NO PRESSURE ULCERS OR HAS NO PRESSURE ULCERS FOR WHICH MOIST WOUND HEALING IS INDICATED.'],
       dtype=object))

(89,
 array([nan, 'NA - NO INPATIENT FACILITY ADMISSION', '1 - HOSPITAL',
        '3 - NURSING HOME', '2 - REHABILITATION FACILITY', '4 - HOSPICE'],
       dtype=object))

<div class="alert alert-danger" role="alert">
    <b>View values of a particular Feature Column</b>
</div>

In [281]:
#df[cols_Oevents].iloc[: , 35]
df_events_int.iloc[: , 36].head()

0      NaN
1      NaN
2      NaN
3      NaN
4    150.0
Name: M1060_WEIGHT_B, dtype: float64

<div class="alert alert-danger" role="alert">
    <b>Get DataTypes of Each Feature Column</b>
</div>

In [282]:
df_events_int.dtypes

index                               int64
M0100_ASSMT_REASON                 object
M0110_EPISODE_TIMING               object
M0140_ETHNIC_AI_AN                 object
M0140_ETHNIC_ASIAN                 object
M0140_ETHNIC_BLACK                 object
M0140_ETHNIC_HISP                  object
M0140_ETHNIC_NH_PI                 object
M0140_ETHNIC_WHITE                 object
M1000_DC_IPPS_14_DA                object
M1000_DC_IRF_14_DA                 object
M1000_DC_LTC_14_DA                 object
M1000_DC_LTCH_14_DA                object
M1000_DC_NONE_14_DA                object
M1000_DC_OTH_14_DA                 object
M1000_DC_PSYCH_14_DA               object
M1000_DC_SNF_14_DA                 object
M1028_ACTV_DIAG_DM                 object
M1028_ACTV_DIAG_NOA                object
M1028_ACTV_DIAG_PVD_PAD            object
M1030_THH_ENT_NUTRITION            object
M1030_THH_IV_INFUSION              object
M1030_THH_NONE_ABOVE               object
M1030_THH_PAR_NUTRITION           

In [283]:
df_events_int.columns

Index(['index', 'M0100_ASSMT_REASON', 'M0110_EPISODE_TIMING',
       'M0140_ETHNIC_AI_AN', 'M0140_ETHNIC_ASIAN', 'M0140_ETHNIC_BLACK',
       'M0140_ETHNIC_HISP', 'M0140_ETHNIC_NH_PI', 'M0140_ETHNIC_WHITE',
       'M1000_DC_IPPS_14_DA', 'M1000_DC_IRF_14_DA', 'M1000_DC_LTC_14_DA',
       'M1000_DC_LTCH_14_DA', 'M1000_DC_NONE_14_DA', 'M1000_DC_OTH_14_DA',
       'M1000_DC_PSYCH_14_DA', 'M1000_DC_SNF_14_DA', 'M1028_ACTV_DIAG_DM',
       'M1028_ACTV_DIAG_NOA', 'M1028_ACTV_DIAG_PVD_PAD',
       'M1030_THH_ENT_NUTRITION', 'M1030_THH_IV_INFUSION',
       'M1030_THH_NONE_ABOVE', 'M1030_THH_PAR_NUTRITION',
       'M1033_HOSP_RISK_5PLUS_MDCTN', 'M1033_HOSP_RISK_COMPLIANCE',
       'M1033_HOSP_RISK_CRNT_EXHSTN', 'M1033_HOSP_RISK_HSTRY_FALLS',
       'M1033_HOSP_RISK_MLTPL_ED_VISIT', 'M1033_HOSP_RISK_MLTPL_HOSPZTN',
       'M1033_HOSP_RISK_MNTL_BHV_DCLN', 'M1033_HOSP_RISK_NONE_ABOVE',
       'M1033_HOSP_RISK_OTHR_RISK', 'M1033_HOSP_RISK_WEIGHT_LOSS',
       'M1051_PVX_RCVD_AGNCY', 'M1060_HEIGHT_A'

In [284]:
dtale.show(df_events_int)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-danger" role="alert">
    <b>Convert Categorical Values and strings to Integers</b>
</div>

In [285]:
df_events_int = df_events_int.fillna(0)
df_events_int = df_events_int.replace('YES', 2)
df_events_int = df_events_int.replace('NO', 1)


<div class="alert alert-danger" role="alert">
    <b>Extract the integer from the event descriptions</b>
</div>

In [286]:

df_events_int['M0110_EPISODE_TIMING'] = df_events_int['M0110_EPISODE_TIMING'].replace({'EARLY':1 , 'LATER':2, 'NOT APPLICABLE':0, np.nan:0})

In [287]:
mask = df_events_int.select_dtypes(include=['object']).columns
df_events_int.loc[:, mask] = df_events_int.loc[:, mask].apply(lambda x: x[mask.tolist()].str[:2].str.strip(), axis = 1).replace({'NA':0,np.nan:0, 'NAN':0, 'N/':0})

In [288]:
for column in df_events_int:
    if df_events_int[column].dtypes == object:
        df_events_int[column]=df_events_int[column].astype(int)

<div class="alert alert-danger" role="alert">
    <b>Verify All Feature Columns are either Int or Floats</b>
</div>

In [289]:
df_events_int.dtypes

index                               int64
M0100_ASSMT_REASON                  int64
M0110_EPISODE_TIMING                int64
M0140_ETHNIC_AI_AN                  int64
M0140_ETHNIC_ASIAN                  int64
M0140_ETHNIC_BLACK                  int64
M0140_ETHNIC_HISP                   int64
M0140_ETHNIC_NH_PI                  int64
M0140_ETHNIC_WHITE                  int64
M1000_DC_IPPS_14_DA                 int64
M1000_DC_IRF_14_DA                  int64
M1000_DC_LTC_14_DA                  int64
M1000_DC_LTCH_14_DA                 int64
M1000_DC_NONE_14_DA                 int64
M1000_DC_OTH_14_DA                  int64
M1000_DC_PSYCH_14_DA                int64
M1000_DC_SNF_14_DA                  int64
M1028_ACTV_DIAG_DM                  int64
M1028_ACTV_DIAG_NOA                 int64
M1028_ACTV_DIAG_PVD_PAD             int64
M1030_THH_ENT_NUTRITION             int64
M1030_THH_IV_INFUSION               int64
M1030_THH_NONE_ABOVE                int64
M1030_THH_PAR_NUTRITION           

<div class="alert alert-danger" role="alert">
    <b>Combine the descriptive columns with the desired events</b>
</div>

In [290]:
colsM = ['pa_id', 'epi_id', 'ceo_id', 'M0090_INFO_COMPLETED_DT']

In [291]:
df_events = pd.concat([df[colsM], df_events_int], axis = 1).reset_index(drop=True).drop_duplicates().drop('index', axis=1, errors='ignore')#level=None, drop=False, inplace=False, col_level=0, col_fill=''

In [292]:
dtale.show(df_events.head())

In [293]:
dtale.show(df_events.head(100))

<div class="alert alert-danger alert-info">
    <b>Convert from Wide to Long format</b>
</div>

In [294]:
df_OASIS = df_events.melt(id_vars=colsM, var_name = 'OASIS')

In [295]:
df_OASIS = df_OASIS.merge(event_mapping_df,left_on='OASIS', right_on='OASIS', how='left')

In [296]:
df_OASIS[df_OASIS.itemID.isna()]


,pa_id,epi_id,ceo_id,M0090_INFO_COMPLETED_DT,OASIS,value,itemID


<div class="alert alert-danger alert-info">
    <b>Convert itemID to Integers (this actually caused problems later on when serializing to JSON!!</b>
</div>

In [297]:
#df_OASIS.fillna({'itemID':0,}, inplace=True)
df_OASIS['itemID'] = df_OASIS['itemID'].apply(np.int64)

np.where(~ItemID.isin([,]))

<div class="alert alert-danger alert-info">
    <b>Rearrange the columns to match MIMIC</b>
</div>

In [298]:
mid = df_OASIS['itemID']
mid2 = df_OASIS['value']
df_OASIS.drop(labels=['itemID', 'value'], axis=1, inplace = True)
df_OASIS.insert(4, 'itemID', mid)
df_OASIS.insert(5, 'value', mid2)

In [299]:
len(df_OASIS)
#df_OASIS.head()
dtale.show(df_OASIS.head(1000))

846479

<div class="alert alert-danger alert-info">
    <b>Export to CSV</b>
</div>

In [301]:
df_OASIS.to_csv(csvStore+'/df_OASIS.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an visits df
</div>

In [302]:
dfv = df.set_index(['pa_id', 'epi_id', 'ceo_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [303]:
dtale.show(df[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop_duplicates())

In [304]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').duplicated(keep=False)

In [305]:
dtale.show(df[df_visits][['pa_id', 'epi_id', 'ceo_id','M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']])

In [306]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [307]:
dtale.show(df_visits) #'epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 

In [308]:
df_visits.to_csv(csvStore+'/df_visits.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an episodes df
</div>

In [309]:
dfe = df.set_index(['pa_id', 'epi_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [310]:
df_episodes = dfe[['mortality','epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 'epi_AdmitType',  'dcc_desc', 'dcr_desc', 'icd_code', 'icd_code_description']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [311]:
int(~pd.isna(df_episodes.epi_DischargeDate).values[0])

1

In [312]:
dtale.show(df_episodes)

In [313]:
df_episodes.to_csv(csvStore+'/df_episode.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

In [314]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3128 entries, 0 to 3127
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   pa_id                 3128 non-null   int64         
 1   DispositionId         3128 non-null   int64         
 2   pa_gender             3128 non-null   object        
 3   year_born             3128 non-null   int64         
 4   M0906_DC_TRAN_DTH_DT  2932 non-null   datetime64[ns]
 5   mortality             3128 non-null   int64         
 6   cedd_dateofdeath      2841 non-null   datetime64[ns]
 7   yod                   3128 non-null   int64         
 8   aod                   3128 non-null   int64         
dtypes: datetime64[ns](2), int64(6), object(1)
memory usage: 220.1+ KB


In [315]:
df_episodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4575 entries, 0 to 4574
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   pa_id                 4575 non-null   int64         
 1   epi_id                4575 non-null   int64         
 2   mortality             4575 non-null   int64         
 3   epi_StartOfEpisode    4575 non-null   datetime64[ns]
 4   epi_EndOfEpisode      4575 non-null   datetime64[ns]
 5   epi_DischargeDate     3557 non-null   datetime64[ns]
 6   epi_AdmitType         4575 non-null   object        
 7   dcc_desc              4571 non-null   object        
 8   dcr_desc              4571 non-null   object        
 9   icd_code              4420 non-null   object        
 10  icd_code_description  4420 non-null   object        
dtypes: datetime64[ns](3), int64(3), object(5)
memory usage: 393.3+ KB


In [316]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9023 entries, 0 to 9022
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   pa_id                    9023 non-null   int64         
 1   epi_id                   9023 non-null   int64         
 2   ceo_id                   9023 non-null   int64         
 3   M0090_INFO_COMPLETED_DT  9023 non-null   datetime64[ns]
 4   M0100_ASSMT_REASON       9023 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 352.6+ KB


In [318]:
df_OASIS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846479 entries, 0 to 846478
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   pa_id                    846479 non-null  int64         
 1   epi_id                   846479 non-null  int64         
 2   ceo_id                   846479 non-null  int64         
 3   M0090_INFO_COMPLETED_DT  846479 non-null  datetime64[ns]
 4   itemID                   846479 non-null  int64         
 5   value                    846479 non-null  float64       
 6   OASIS                    846479 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 51.7+ MB
